## 회귀란?
### -분류의 논리적 확장으로, 숫자로 표현된 일련의 특징으로부터 실수(연속형 변수) 예측 
### -수학적 관점에서 출력값의 수가 무한하기때문에 분류보다 더어려울 수 있음 
### -분류문제의 정확도 개념과는 달리 예측값과 실젯값 사이의 오차를 최적화하는 것을 목표로 함 

## 활용 사례
### -영화 관객수 예측 
#### 예고편, 소셜미디어 공유 횟수 기반으로 향후 얼마나 많은 사람이 영화를 볼지 예측
### -회사 수익 예측
#### 회사의 성장궤도, 시장, 및 계정설을 고려하여 회사의 이익 예측 
### -농작물 수확량 예측 
#### 특정 지역 정보, 1년 동안의 날씨 정보 등을 기반으로 수확량 예측 

## MLlib에서 제공하는 회귀모델
#### 1. 선형회귀
#### 2. 일반화 선형회귀
#### 3. 등위회귀
#### 4. 의사결정트리
#### 5. 랜덤포레스트
#### 6. 그래디언트 부스티드 트리
#### 7. 생존 회귀

## 해당 챕터에서는 아래와 같은 각 모델의 기본사항을 다룰 예정!
#### 1. 모델과 알고리즘에 대한 기본 설명
#### 2. 모델 설정을 위한 하이퍼파라미터 (모델 초기화 방법)
#### 3. 학습 파라미터 (모델 학습에 영향을 주는 파라미터)
#### 4. 예측 파라미터 (예측에 영향을 미치는 파라미터)

## 모델 확장성 
#### MLlib에서 제공하는 회귀모델은 모두 큰 규모의 데이터셋으로 확장 가능
#### 모델의 구성, 시스템 규모, 그리고 기타 요소에 다라 달라짐 

### *최대 특징 수 | 최대 학습 데이터 수 
##### 1. 선형회귀 : 1~1천만개 | 제한 없음
##### 2. 일반화 선형 회귀 : 4,096개 | 제한없음
##### 3. 등위 회귀 : N/A : 수백만건 
##### 4. 의사결정 트리 : 1000개 | 제한 없음 
##### 5. 랜덤 포레스트 : 10000개 | 제한없음
##### 6. 그래디언트 부스티드 트리 : 1000개 | 제한없음
##### 7. 생존 회귀 : 1~천만개 | 제한 없음

In [12]:
#sample data load 
df = spark.read.load("regression")


## 선형 회귀

- 입력 특징들의 선형 조합 (가중치가 곱해진 각 특징을 합한 값)이 가우시안 오차와 함께최종 결과로 산출된다고 가정 
- 선형 가정 : 단순하게 해석 가능하며, 과적합 되기 힘든 모델로 만들어줌 
- 로지스틱 회귀에서처럼  ElasticNet 일반화를 통해 L1 및 L2 일반화 혼합 가능 

### 모델 하이퍼파라미터 
- 로지스틱 회귀와 동일 (26장 참고)

### 학습 하이퍼파라미터
- 로지스틱 회귀와 동일 (26장 참고)


In [41]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression().setMaxIter(5).setRegParam(0.3).setElasticNetParam(0.8)
print(lr.explainParams())
lrModel = lr.fit(df)

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0, current: 0.8)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxIter: max number of iterations (>= 0). (default: 100, current: 5)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0, current: 0.3)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (default: auto)
standardization: whether to standardize the tra

- summary : 여러개의 필드로 구성된 요약 객체 반환 
    1. 잔차 : 선형 회귀모델로부터 구한 예측값과 실젯값과의 차이 
    2. 객체 히스토리 : 반복 학습 과정마다 모델 학습이 어떻게 진행되는지 
    3. 평균 제곱근 오차 : 예측값과 실젯값 사이의 거리 계산하여 모델의 성능을 측정하는 척도 
    4. R-제곱변수(r2) : 모델에 의해 설명되는 예측 변수의 분산 비율 

df.show(20)

In [48]:
##평가 지표
summary = lrModel.summary
summary.residuals.show()
print (summary.totalIterations)
print (summary.objectiveHistory)
print (summary.rootMeanSquaredError)
print (summary.r2) #1에 가까울수록 회귀 모델이 데이터를ㅇ 더 잘 설명 (결정계수)



+--------------------+
|           residuals|
+--------------------+
|  0.1280504658561019|
|-0.14468269261572098|
| -0.4190383262242056|
| -0.4190383262242056|
|  0.8547088792080308|
+--------------------+

6
[0.5000000000000001, 0.4315295810362787, 0.3132335933881021, 0.312256926665541, 0.3091506081983029, 0.3091505893348027]
0.47308424392175985
0.720239122691221


## 일반화 선형 회귀

- 다른 회귀 모델에 비해 더 세밀하게 제어할 수 있음 
- 가우스 (선형 회귀), 이항 (로지스틱 회귀), 푸아송 (푸아송 회귀) 그리고 감마 (감마 회귀) 등 다양한 분포 집합으로부터 예상되는 노이즈 분포를 선택할 수 있음 
- 선형 함수와 분포 함수의 평균 사이의 관계를 지정하는 링크 함수 설정을 지원함 
- 가우스 : Identity.Log.Inverse
- 이항 : Logit.Probit.ClogLog
- 푸아송 : Log.Identity.Sqrt
- 감마 : Inverse.Identity.Log
- 트위디 : 파워 링크 함수

### 모델 하이퍼 파라미터 
1. family : 모델에서 사용할 오차 분포에 대한 설명으로, 가우스, 이항, 푸아송, 감마, 그리고 트위디가 있음
2. link : 선형 함수와 분포 함수의 평균 사이의 관계를 지정하는 링크 함수의 이름. 지원되는 옵션 : cloglog, probit, logit, inverse, sqrt, identity, 그리고 log (기본값은 identity) 
3. solver : 최적화에 사용되는 해찾기 알고리즘. 현재 지원되는 유일한 solver는 irls (반복재가중최소제곱) 
4. variancePower : 분포의 분산과 평균 사이의 관계를 특징짓는 트위디 분포의 분산 함수에서의 검증력. 트위디에만 적용되며 지원되는 값은 0과 [1,무한). 기본값 =0 
5. linkPower : 트위디 family의 파워 링크 함수 색인

### 학습 하이퍼파라미터 
- 로지스틱 회귀와 동일 (26장 참고)

### 예측 파라미터
- 하나의 예측 파라미터 제공 
- linePredictionCol : 예측에 적용할 각 링크 함수를 정의하는 컬럼명 

In [47]:
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression()\
  .setFamily("gaussian")\
  .setLink("identity")\
  .setMaxIter(10)\
  .setRegParam(0.3)\
  .setLinkPredictionCol("linkOut")
print (glr.explainParams())
glrModel = glr.fit(df)


family: The name of family which is a description of the error distribution to be used in the model. Supported options: gaussian (default), binomial, poisson, gamma and tweedie. (default: gaussian, current: gaussian)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
link: The name of link function which provides the relationship between the linear predictor and the mean of the distribution function. Supported options: identity, log, inverse, logit, probit, cloglog and sqrt. (current: identity)
linkPower: The index in the power link function. Only applicable to the Tweedie family. (undefined)
linkPredictionCol: link prediction (linear predictor) column name (current: linkOut)
maxIter: max number of iterations (>= 0). (default: 25, current: 10)
offsetCol: The offset column name. If this is not set or empty, we treat all instance offsets as 0.0 (undefined)
predictionCol: pred

### 평가지표
- R squared : 결정 계수. 적합의 척도
- The residuals : 관측 레이블과 예측값의 차이 

In [63]:
##평가 지표
summary = glrModel.summary
summary


# st_val = sum(map(lambda x: np.power(x,2),y-np.mean(y))) 
# sse_val = sum(map(lambda x: np.power(x,2),m1.resid_response)) 
# r2 = 1.0 - sse_val/sst_val

Coefficients:
    Feature Estimate Std Error T Value P Value
(Intercept)   0.0867    1.2210  0.0710  0.9549
 features_0   0.3661    0.7686  0.4764  0.7170
 features_1   0.0466    0.1380  0.3377  0.7927
 features_2   0.1831    0.3843  0.4764  0.7170

(Dispersion parameter for gaussian family taken to be 0.8466)
    Null deviance: 4.0000 on 1 degrees of freedom
Residual deviance: 0.8466 on 1 degrees of freedom
AIC: 15.3094

## 의사결정트리 
- 분류에 적용된 의사결정 트리와 상당히 유사하게 작동
- 차이점은 회귀의 의사결정트리는 분류의 레이블 대신 말단 노드마다 하나의 숫자를 출력 
- 함수를 모델링하기 위해 계수를 학습하는 대신 단순히 수치를 예측하는 트리를 만듦
- 선형 회구와 달리 입력 데이터의 비선형 패턴을 예측할 수 있기 때문에 유의미하나, 과적합될 수 있는 위험이 높음 

### 모델 하이퍼 파라미터 
- impurity 파라미터의 경미한 변경을 제외하고는 분류와 동일 
- impurity: 모델이 특정 값을 갖는 특정 말단 노드에서 분할되어야 하는지 아니면 그대로 유지되어야 하는지 여부에 대한 평가지표를 의미. 회귀트리에 지원되는 유일한 평가지표는 '분산'

### 학습 하이퍼 파라미터 
- 26.4.2절 '학습 파라미터' 참조 -> 분류와 동일

In [67]:
from pyspark.ml.regression import DecisionTreeRegressor
dtr = DecisionTreeRegressor()
print (dtr.explainParams())
dtrModel = dtr.fit(df)

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: variance (default: variance)
labelCol: label column name. (default: label)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. 

## 랜덤포레스트와 그래디언트 부스티드 트리 
- 랜덤 포레스트와 그래디언트 부스티드 트리 모델은 분류 및 회귀 모델에 모두 적용 가능 
- 의사결정트리와 동일한 개념이지만 회귀를 수행하는 여러개의 트리 학습
- 랜덤 포레스트는 상관관계가 없는 다양한 의사결정트리가 학습되고 평균화되며, 그래디언트에서는 개별 트리가 학습될 때 별도의 가주잋 부여 
- 순도(purity) 척도를 제외하면 일반 분류모델과 동일한 모델/학습 파라미터 가짐

In [71]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
rf =  RandomForestRegressor()
print (rf.explainParams())
rfModel = rf.fit(df)
gbt = GBTRegressor()
print (gbt.explainParams())
gbtModel = gbt.fit(df)

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 

## 고급 방법론
1. 생존 회귀 (가속 수명 시간 모델)
- 통제된 실험 환경에서 생존 분석으로 개인의 생존율 이해 
- 생존 시간의 로그를 모델화 하는 가속 수명 시간 모델을 구현 
- 스파크에서 잘 알려진 콕스 비례 위험 모델 대신 생존 회귀 모델을 사용함 (콕스 비례 위험 모델은 준모수적이고, 대용량 데이터셋 확장이 어려움)
- 중도전달 변수를 도입해야 함 (피험자가 실험을 그만두는 경우, 연구자에게 상태가 전달되지 않은 피험자의 최종 상태를 추정할 수 없기 때문에 사용)
    - 생존 분석에서 손실된 데이터를 처리하는 방법 

2. 등위 회귀 
- 특별한 요구사항이 존재하는 또 다른 특수한 회귀 모델 
- 항상 단조롭게 증가하는 구간적 선형 함수를 지정한다. (즉 감소 불가능)
- 데이터가 우상향하는 추이를 보이면 모델이 적절하다는 의미를 가짐. 입력값이 변화함에 따라 우상향 추이를 보이지 않는다면 올바른 모델이라 할 수 없음 

## 평가기와 모델 튜닝 자동화 
- 분류 문제에서 살펴본 코어 모델 튜닝 기능을 동일하게 제공 
- 평가기를 지정하고 최적화할 수 있는 지표 선택 후, 파이프라인을 학습시켜 파라미터를 튜닝함 
- 회귀 분석을 위한 평가기는 RegressionEvaluator라고 부르며, 일반적인 회귀성과측정 평가지표를 최적화시킴
- 예측값과 실젯값을 나타내는 두개의 컬럼 제공 
- 최적화를 위해 지원되는 평가지표 : 평균제곱근오차(rsme), 평균제곱오차(mse), 결정계수(r2), 평균절대오차(mae)

In [80]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
glr = GeneralizedLinearRegression().setFamily("gaussian").setLink("identity")
pipeline = Pipeline().setStages([glr])
params = ParamGridBuilder().addGrid(glr.regParam, [0.0, 0.5, 1.0]).build()
evaluator = RegressionEvaluator()\
  .setMetricName("rmse")\
  .setPredictionCol("prediction")\
  .setLabelCol("label")
cv = CrossValidator()\
  .setEstimator(pipeline)\
  .setEvaluator(evaluator)\
  .setEstimatorParamMaps(params)\
  .setNumFolds(2) # should always be 3 or more but this dataset is small
model = cv.fit(df)


## 평가지표 
- 평가기를 사용하면 하나의 특정한 평가지표에 따라 모델을 평가하고 적합시킬 수 있음 
- RegressionMetrics 객체를 통해 추가로 다양한 회귀 평가지표를 검토할 수 있음
    - (예측, 레이블 ) 쌍의 RDD를 기반으로 동작 

In [91]:
from pyspark.mllib.evaluation import RegressionMetrics
out = model.transform(df)\
  .select("prediction", "label").rdd.map(lambda x: (float(x[0]), float(x[1])))
metrics = RegressionMetrics(out)
print ("MSE: " + str(metrics.meanSquaredError))
print ("RMSE: " + str(metrics.rootMeanSquaredError))
print ("R-squared: " + str(metrics.r2))
print ("MAE: " + str(metrics.meanAbsoluteError))
print ("Explained variance: " + str(metrics.explainedVariance))

MSE: 0.18038162667943697
RMSE: 0.42471358193426895
R-squared: 0.7745229666507039
MAE: 0.3277600580484718
Explained variance: 0.4448396208322792
